### Install the required libraries

In [ ]:
!pip install howso-engine

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 449.6/449.6 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 48.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.6/72.6 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.6/380.6 kB 47.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 105.7 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.7
    Uninstalling urllib3-2.0.7:
      Successfully uninstalled urllib3-2.0.7
  Attempting uninstall: pyjwt
    Found existing installation: PyJWT 2.3.0
    Uninstalling PyJWT-2.3.0:
      Successfully uninstalled PyJWT-2.3.0


### Generating the synthetic data

In [ ]:
from howso import engine
from howso.engine import Trainee
from howso.utilities import infer_feature_attributes

import pandas as pd
import os

def howso_runner(df, feature_attr):
    features = infer_feature_attributes(df)

    for key in feature_attr:
        for subkey in feature_attr[key]:
            if isinstance(feature_attr[key][subkey], dict):
                for subsubkey in feature_attr[key][subkey]:
                    features[key][subkey][subsubkey] = feature_attr[key][subkey][subsubkey]
            else:
                features[key][subkey] = feature_attr[key][subkey]

    t = Trainee(
        features=features,
        overwrite_existing=True
    )

    t.train(df)
    t.analyze()

    # if f_name in ["profb", "breast_cancer", "tic_tac_toe"]:
    synth = t.react(action_features=df.columns.tolist(),
                    desired_conviction=5,
                    generate_new_cases="no",
                    num_cases_to_generate=16*len(df)) # Change the number to denote the size of the synthetic data you want. In this case it generates 16 times the len of original data
    # else:
    #     synth = t.react(action_features=df.columns.tolist(),
    #                     desired_conviction=5,
    #                     generate_new_cases="always",
    #                     num_cases_to_generate=len(df))

    return synth['action']

filepaths = ['/content/ivy-1.1_Copy.csv','/content/ivy-1.4_Copy.csv','/content/ivy-2.0_Copy.csv'] # Replace these with the paths of the data files you want to use
for f_path in filepaths:
  filename = f_path.split('/')[-1][:-4]
  print(filename)
  df = pd.read_csv(f_path)
  print("Working on file -> ", filename)
  # print(df)
  # feature_attr = {"cars_per_hour": {"type": "ordinal",
  #                                   "original_type": {"data_type": "adb"},
  #                                   "non_sensitive": True},
  #                 "day": {"type": "ordinal",
  #                         "original_type": {"data_type": "bde", "size": 16},
  #                         "non_sensitive": True}
  #                 }
  feature_attr = {}
  new_df = howso_runner(df, feature_attr)
  print(new_df.shape)
  print('Data genaration completed')
  print("------------------------------")

  # This code will save the generated file as a csv file
  new_file = filename + "-16x.csv"
  new_df.to_csv(new_file, index = False)

ivy-1.1_Copy
Working on file ->  ivy-1.1_Copy
(1776, 21)
Data genaration completed
------------------------------
ivy-1.4_Copy
Working on file ->  ivy-1.4_Copy
(3856, 21)
Data genaration completed
------------------------------
ivy-2.0_Copy
Working on file ->  ivy-2.0_Copy
(5632, 21)
Data genaration completed
------------------------------
